In [345]:
import pandas as pd
from sklearn import linear_model
from sklearn.utils import shuffle
from sklearn.metrics import mean_absolute_error

In [346]:
# wczytuję dane
df = pd.read_csv('./data/california_housing.csv')
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,Unnamed: 9
count,3000.000000,3000.00000,3000.000000,3000.000000,3000.000000,3000.000000,3000.00000,3000.000000,3000.00000,0.0
mean,-119.589200,35.63539,28.845333,2599.578667,529.950667,1402.798667,489.91200,3.807272,205846.27500,NaN
std,1.994936,2.12967,12.555396,2155.593332,415.654368,1030.543012,365.42271,1.854512,113119.68747,NaN
min,-124.180000,32.56000,1.000000,6.000000,2.000000,5.000000,2.00000,0.499900,22500.00000,NaN
25%,-121.810000,33.93000,18.000000,1401.000000,291.000000,780.000000,273.00000,2.544000,121200.00000,NaN
50%,-118.485000,34.27000,29.000000,2106.000000,437.000000,1155.000000,409.50000,3.487150,177650.00000,NaN
75%,-118.020000,37.69000,37.000000,3129.000000,636.000000,1742.750000,597.25000,4.656475,263975.00000,NaN
max,-114.490000,41.92000,52.000000,30450.000000,5419.000000,11935.000000,4930.00000,15.000100,500001.00000,NaN


In [347]:
# usuwam pustą kolumnę, i kolumnę ocean_proximity, poniewasz zawiera niepełne dane, nie wiadomo jak należy intepretować nany
df.drop(columns=['Unnamed: 9', 'ocean_proximity'], inplace=True)
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27,3885,661,1537,606,6.6085,344700
1,-118.30,34.26,43,1510,310,809,277,3.5990,176500
2,-117.81,33.78,27,3589,507,1484,495,5.7934,270500
3,-118.36,33.82,28,67,15,49,11,6.1359,330000
4,-119.67,36.33,19,1241,244,850,237,2.9375,81700
...,...,...,...,...,...,...,...,...,...
2995,-119.86,34.42,23,1450,642,1258,607,1.1790,225000
2996,-118.14,34.06,27,5257,1082,3496,1036,3.3906,237200
2997,-119.70,36.30,10,956,201,693,220,2.2895,62000
2998,-117.12,34.10,40,96,14,46,14,3.2708,162500


In [348]:
# liczę średnie, aby zmienne były niezależne od siebie
df['avg_rooms'] = df['total_rooms'] / df['households']
df['avg_bedrooms'] = df['total_bedrooms'] / df['households']
df['avg_population'] = df['population'] / df['households']
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,avg_rooms,avg_bedrooms,avg_population
0,-122.05,37.37,27,3885,661,1537,606,6.6085,344700,6.410891,1.090759,2.536304
1,-118.30,34.26,43,1510,310,809,277,3.5990,176500,5.451264,1.119134,2.920578
2,-117.81,33.78,27,3589,507,1484,495,5.7934,270500,7.250505,1.024242,2.997980
3,-118.36,33.82,28,67,15,49,11,6.1359,330000,6.090909,1.363636,4.454545
4,-119.67,36.33,19,1241,244,850,237,2.9375,81700,5.236287,1.029536,3.586498
...,...,...,...,...,...,...,...,...,...,...,...,...
2995,-119.86,34.42,23,1450,642,1258,607,1.1790,225000,2.388797,1.057661,2.072488
2996,-118.14,34.06,27,5257,1082,3496,1036,3.3906,237200,5.074324,1.044402,3.374517
2997,-119.70,36.30,10,956,201,693,220,2.2895,62000,4.345455,0.913636,3.150000
2998,-117.12,34.10,40,96,14,46,14,3.2708,162500,6.857143,1.000000,3.285714


In [349]:
features = ['longitude', 'latitude', 'housing_median_age', 'avg_population', 'households', 'median_income', 'avg_rooms', 'avg_bedrooms']
# mieszam dataset
df = shuffle(df)
X, y = df[features], df['median_house_value']
# train/test = 2500/500
X_train, X_test, y_train, y_test = X[500:], X[:500], y[500:], y[:500]
X_train, X_test, y_train, y_test

(      longitude  latitude  housing_median_age  avg_population  households  \
 1075    -122.28     38.29                  19        1.737500          80   
 582     -117.87     33.74                  16        5.119681         376   
 433     -116.84     33.08                  15        3.204348         460   
 167     -121.56     38.26                  43        3.171975         314   
 665     -118.14     33.89                  33        2.516312         705   
 ...         ...       ...                 ...             ...         ...   
 2748    -118.07     34.17                  34        2.694346         566   
 1549    -117.77     33.72                   9        2.944444         324   
 2156    -120.79     38.43                  40        2.551402         214   
 766     -118.41     34.18                  35        2.172414         406   
 1011    -120.32     37.29                  38        4.267857         112   
 
       median_income  avg_rooms  avg_bedrooms  
 1075         

In [350]:
# regresja liniowa
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)
r2 = reg.score(X_test, y_test)
mae = mean_absolute_error(y_test, reg.predict(X_test))
r2, mae

(0.6070097265731218, 51010.71208936498)

średni bład wynosi ~50k, średia wartość domu to ~200k, więc ten model nie sprawdza się zbyt dobrze